КОРОЧЕ
В этом ноутбуке я пытался обучить ast - трансформер для аудио классификации, но он жрет овер дохера, поэтому выходы: CNN / CNN + attention

In [2]:
DATA_PATH = './data'
TRAIN_DATA_PATH = f'{DATA_PATH}/train'
TASK_DATA_PATH = f'{DATA_PATH}/train'
TRAIN_FILENAME = f'{DATA_PATH}/train_gt.csv'
TASK_FILENAME = f'{DATA_PATH}/test.csv'

In [3]:
import pandas as pd
from transformers import AutoFeatureExtractor, ASTForAudioClassification, Trainer, ASTConfig, ASTModel, TrainingArguments, ASTFeatureExtractor
from datasets import Dataset, load_dataset, Audio
import torch

/opt/miniconda3/envs/tune-ast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv(TRAIN_FILENAME, header=None, names=['audio', 'label'])[:100]#RM
df['audio'] = TRAIN_DATA_PATH + "/" + df['audio']

dataset = Dataset.from_pandas(df).cast_column("audio", Audio())
sampling_rate = dataset.features["audio"].sampling_rate

In [5]:
model_config = ASTConfig(

)

model = ASTForAudioClassification(model_config)
feature_extractor = ASTFeatureExtractor()

/opt/miniconda3/envs/tune-ast/lib/python3.11/site-packages/transformers/audio_utils.py:297: UserWarning: At least one mel filter has all zero values. The value for `num_mel_filters` (128) may be set too high. Or, the value for `num_frequency_bins` (256) may be set too low.
  warnings.warn(


In [6]:
# Preprocess the dataset
def preprocess_function(examples):
    audio = examples["audio"]  # Adjust based on your dataset structure
    inputs = feature_extractor(audio["array"], return_tensors='np', feature_size=-1, sampling_rate=16000) # Probably not 16000
    inputs['input_values'] = inputs['input_values'].reshape(1024, 128)
    return inputs

# Metrics computing while fine tuning
def compute_metrics(p):
    predictions, labels = p

    print(predictions, labels)

    return {
        "precision": 0,
        "recall": 0,
        "f1": 0,
        "accuracy": 0,
    }

In [7]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"]).train_test_split(test_size=0.3)

Map: 100%|██████████| 100/100 [00:01<00:00, 57.18 examples/s]


In [8]:
training_arguments = TrainingArguments(
    output_dir="aiijc_ast",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    # data_collator=data_collator,
    compute_metrics=None,
)

trainer.train()

/opt/miniconda3/envs/tune-ast/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 17.78 GB, other allocations: 2.64 MB, max allowed: 18.13 GB). Tried to allocate 1.05 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).